In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import warnings

warnings.filterwarnings('ignore')
sns.set_theme()

In [16]:
def make_group(event: str):
  # Посещение корзины с последующим оформлением заказа
  g1 = ['Посещение корзины с добавленными товарами', 'Ввод промокода']

  # 1е шаги оформления
  g2 = ['Клик на кнопку Перейти к оформлению', 'Выбор радиобаттона eSIM', 'Выбор радиобаттона обычной SIM']

  # Изменение тарифа
  g3 = ['Выбор другого тарифа или номера в корзине']

  # Ввод личных данных
  g4 = ['Ввод контактного номера', 'Ввод имени', 'Ввод email',
        'Клик на кнопку Продолжить 1 (Экран ввода контактных данных)', 'Клик на кнопку Продолжить 2 (Экран выбора региона)']

  # Оформление доставки
  g5 = ['Клик на кнопку Продолжить 3 (Экран выбора способа получения)', 'Клик на кнопку Продолжить 4 (Экран выбора адреса доставки/самовывоза)']

  # Заполнение паспортных данных
  g6 = ['Включение блока паспортных данных', 'Успешное заполнение всех полей паспортных данных']

  # Покупка
  g7 = ['Покупка']

  if event in g1:
    return 1
  elif event in g2:
    return 2
  elif event in g3:
    return 3
  elif event in g4:
    return 4
  elif event in g5:
    return 5
  elif event in g6:
    return 6
  elif event in g7:
    return 7


In [27]:
def good_events(arr): 
    ''' Функция убирает те действия, которые повторяются. '''
    ans = [arr[0]]  
    for i in range(1, len(arr)):
        if arr[i] != arr[i-1]:
            ans.append(arr[i])
    return ans


def do_voronka(df, delet_koef=0):
    ''' Функция принимает на вход датафрейм и создаёт пользовательские сценарии и считает их кол-во. 
    Есть условный параметри delet_koef - он удаляет те сценарии, которые встерчаются < delet_koef'''
    df = df[['client_id', 'session_id','event_time', 'event_name']]
    df = df.sort_values('event_time').drop('event_time', axis=1)
    df = df.groupby(['client_id', 'session_id'], as_index=False).agg(lambda x: tuple(good_events( x.to_list()) + [0]))
    df['summ'] = 1
    df = df.drop(['client_id', 'session_id'], axis=1)
    df = df.groupby('event_name', as_index=False).agg('sum')[(df['summ'] > delet_koef) | (df.event_name.apply(lambda x: 5 in x))] 
    return df

def do_graph(df, depth=100, delet_koef=0):
    G = nx.DiGraph()
    df = df[['client_id', 'session_id','event_time', 'event_name']]
    df = df.sort_values('event_time').drop('event_time', axis=1)
    df = df.groupby(['client_id', 'session_id'], as_index=False).agg(lambda x: tuple(good_events(x.to_list()) + [0]))
    df['summ'] = 1
    df = df.drop(['client_id', 'session_id'], axis=1)
    df = df.groupby('event_name', as_index=False).agg('sum')
    for i in range(df.shape[0]):
        event = df.iloc[i]
        events = event.event_name 
        summa = event.summ
        for k in range(len(events)-1):
            first_node = str(events[k]) + '_' + str(k)
            second_node = str(events[k+1]) + '_' + str(k+1)
            if not G.has_node(first_node):
                G.add_node(first_node)
            if not G.has_node(second_node):
                G.add_node(second_node)
            weight = summa
            if G.has_edge(first_node, second_node):
                weight += G.get_edge_data(first_node, second_node)['weight']
            G.add_edge(first_node, second_node, weight=weight)
                        
    return G

def do_voronka_uliana(df):
    ''' Функция принимает на вход датафрейм и создаёт словарь, где описывается кол-во последовательностей действий'''
    moves = {i: {k: 0 for k in range(0, 8) if k != i} for i in range(1, 8)}
    df = df[['client_id', 'session_id','event_time', 'event_name']]
    df = df.sort_values('event_time').drop('event_time', axis=1)
    df = df.groupby(['client_id', 'session_id'], as_index=False).agg(lambda x: tuple(good_events(x.to_list()) + [0]))
    df['summ'] = 1
    df = df.drop(['client_id', 'session_id'], axis=1)
    df = df.groupby('event_name', as_index=False).agg('sum')
    for i in range(df.shape[0]):
        event = df.iloc[i]
        events = event.event_name 
        summa = event.summ
        for k in range(len(events)-1):
            first = events[k]
            second = events[k+1]
            moves[first][second] += summa
    return moves

In [18]:
def do_voronka_final(df):
    df = df[['client_id', 'session_id','event_time', 'event_name']]
    df = df.sort_values('event_time').drop('event_time', axis=1)
    df = df.groupby(['client_id', 'session_id'], as_index=False).agg(lambda x: tuple(good_events(x.to_list())))
    df = df.groupby('client_id').agg(lambda x: tuple(x.to_list()))
    return df

In [19]:
df = pd.read_csv("tele2_dataset.csv", parse_dates=['date', 'event_time'], )
sorted_df = df[['client_id', 'session_id']].groupby('session_id', as_index=False).agg(lambda x: tuple(set(x.to_list())))
sorted_df = sorted_df[sorted_df['client_id'].apply(lambda x: len(x) > 1)]
sorted_df_list = sorted_df['session_id'].to_list()

df = df[df.session_id.apply(lambda x: x not in sorted_df_list)]

In [20]:
df = df.sort_values('event_time')
df['event_name'] = df.event_name.apply(lambda x: make_group(x))

In [21]:
    colliz = df[['client_id', 'session_id']].groupby('session_id', as_index=False).agg(lambda x: tuple(set(x.to_list())))
    colliz = df[['client_id', 'session_id']].groupby('session_id', as_index=False).agg(lambda x: tuple(set(x.to_list())))[df[['client_id', 'session_id']].groupby('session_id', as_index=False).agg(lambda x: tuple(set(x.to_list())))['client_id'].apply(lambda x: len(x) == 1)]
    colliz

,session_id,client_id
0,141972659907373,"(1043516595.1666613673,)"
1,168600804989959,"(1245916659.1669233,)"
2,456173673257459,"(991426336.166929,)"
3,938695947723453,"(1594686043.166956,)"
4,1186551691689791,"(1371117828.1666691,)"
...,...,...
30437,9219115026099903349,"(1142792924.1668308,)"
30438,9220217623014508991,"(2022388297.1667209,)"
30439,9220851645177312063,"(1939339809.1669388,)"
30440,9222232445890283069,"(1274334158.1668792,)"


# Условные обозначения 

0 - конец сессии

1 - Посещение корзины с последующим оформлением заказа

2 - Ввод личных данных

3 - Оформление заказа

4 - Заполнение паспортных данных

5 - покупка

In [23]:
#do_voronka_final(df)

In [24]:
viborka = df[(df['region_name'] == 'MSK') & (df['rfm_index'] > 222) & (df['campaign'] != '(not set)')]

In [25]:
voronka = do_voronka(df)
voronka.tail(40)

,event_name,summ
1601,"(3, 7, 0)",2
1602,"(4, 0)",30
1603,"(4, 1, 0)",3
1604,"(4, 1, 2, 1, 2, 3, 2, 1, 4, 5, 4, 2, 5, 7, 0)",1
1605,"(4, 1, 2, 4, 1, 0)",1
1606,"(4, 1, 2, 4, 1, 3, 2, 4, 5, 7, 0)",1
1607,"(4, 1, 3, 2, 4, 5, 7, 0)",3
1608,"(4, 1, 4, 1, 2, 4, 5, 6, 7, 0)",1
1609,"(4, 1, 4, 5, 1, 3, 2, 0)",1
1610,"(4, 1, 4, 5, 7, 0)",1


In [28]:
voronka = do_voronka_uliana(df)
voronka

{1: {0: 15743, 2: 10027, 3: 10784, 4: 671, 5: 172, 6: 142, 7: 132},
 2: {0: 1959, 1: 1182, 3: 1057, 4: 11464, 5: 213, 6: 3, 7: 71},
 3: {0: 3972, 1: 2795, 2: 5494, 4: 33, 5: 2, 6: 0, 7: 4},
 4: {0: 971, 1: 827, 2: 120, 3: 147, 5: 10420, 6: 13, 7: 78},
 5: {0: 789, 1: 997, 2: 63, 3: 115, 4: 322, 6: 242, 7: 8327},
 6: {0: 52, 1: 71, 2: 3, 3: 5, 4: 16, 5: 29, 7: 227},
 7: {0: 6956, 1: 1865, 2: 17, 3: 4, 4: 0, 5: 5, 6: 0}}

In [13]:
len('Клик на кнопку Продолжить 4 (Экран выбора адреса доставки/самовывоза)')

69

In [15]:
graph = do_graph(viborka)